In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
from tqdm import tqdm
import gc

from sklearn import set_config
from sklearn.base import clone
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm.callback import early_stopping, log_evaluation

sns.set_theme(style = 'white', palette = 'viridis')
pal = sns.color_palette('viridis')

pd.set_option('display.max_rows', 100)
set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None

In [2]:
DATA_DIR = '/kaggle/input/optiver-trading-at-the-close'
# DATA_DIR = 'input/'

In [3]:
train = pd.read_csv(f'{DATA_DIR}/train.csv').drop(['row_id', 'time_id'], axis = 1)
test = pd.read_csv(f'{DATA_DIR}/example_test_files/test.csv').drop(['row_id', 'time_id'], axis = 1)

# Preparation

In [13]:
X = train[~train.target.isna()]
y = X.pop('target')

seed = 42
tss = TimeSeriesSplit(10)

os.environ['PYTHONHASHSEED'] = '42'
tf.keras.utils.set_random_seed(seed)

In [5]:
def imbalance_calculator(x):
    
    x_copy = x.copy()
    
    x_copy['imb_s1'] = x.eval('(bid_size - ask_size) / (bid_size + ask_size)')
    x_copy['imb_s2'] = x.eval('(imbalance_size - matched_size) / (matched_size + imbalance_size)')
    
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            if i>j:
                x_copy[f'{a}_{b}_imb'] = x.eval(f'({a} - {b}) / ({a} + {b})')
                    
    for i,a in enumerate(prices):
        for j,b in enumerate(prices):
            for k,c in enumerate(prices):
                if i>j and j>k:
                    max_ = x[[a,b,c]].max(axis=1)
                    min_ = x[[a,b,c]].min(axis=1)
                    mid_ = x[[a,b,c]].sum(axis=1)-min_-max_

                    x_copy[f'{a}_{b}_{c}_imb2'] = (max_-mid_)/(mid_-min_)
    
    return x_copy

ImbalanceCalculator = FunctionTransformer(imbalance_calculator)

In [6]:
class LGBMRegressorCV:
    
    
    def __init__(self, params, cv, n_estimators=1000, early_stopping_rounds=30):
        
        self.params = params
        self.cv = cv
        self.n_estimators = n_estimators
        self.early_stopping_rounds = early_stopping_rounds
        
        self.models = []
        self.best_val_score = float('inf')
        self.val_scores = []
        self.train_scores = []
        self.best_model = None
        
        
    def fit(self, X, y):
        for train_index, test_index in tqdm(self.cv.split(X, y)):

            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            model = LGBMRegressor(**self.params, n_estimators=self.n_estimators)
            model.fit(
                X_train, y_train, 
                eval_set=[(X_test, y_test)], 
                eval_metric='mae',
                callbacks=[early_stopping(self.early_stopping_rounds), log_evaluation(period=10)]
            )

            self.models.append(model)
            self.train_scores.append(mean_absolute_error(y_train, model.predict(X_train)))
            self.val_scores.append(mean_absolute_error(y_test, model.predict(X_test)))

            # Find the best model based on validation score
            if self.val_scores[-1] < self.best_val_score:
                self.best_val_score = self.val_scores[-1]
                self.best_model = clone(model)

        return self
    
    
    def predict(self, X):
        # Use best model to predict
        return self.best_model.predict(X)
    
    
    def get_best_model_params(self):
        # Get parameters of the best model
        return self.best_model.get_params()


# Grid Search

In [ ]:
tss = TimeSeriesSplit(n_splits=5)

# Define parameter grid for Grid Search
param_grid = {
#     'lgbm__max_depth': [-1, 3, 5, 7, 9],
    'lgbm__n_estimators': [100, 300, 500, 800, 1000],
    'lgbm__min_child_weight': [0.001, 0.003, 0.01, 0.03, 0.1],
    'lgbm__subsample': [0.6, 0.8, 1.0],
    'lgbm__colsample_bytree': [0.6, 0.8, 1.0],
#     'lgbm__reg_alpha': [0, 0.1, 0.3, 1.0],
#     'lgbm__reg_lambda': [0, 0.1, 0.3, 1.0],
    'lgbm__boosting_type': ['gbdt', 'dart']
}

# Initialize pipeline with Imbalance Calculator and LightGBM Regressor
lgbm_model = LGBMRegressor(objective='mae')
pipeline = Pipeline(steps=[('imbalance_calculator', ImbalanceCalculator), ('lgbm', lgbm_model)])

# Initialize GridSearchCV with TimeSeriesSplit
grid_search_tss = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=tss, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_error')

# Fit the model
grid_search_tss.fit(X, y)

# Get the best parameters
best_params_tss = grid_search_tss.best_params_

# Extract the best LightGBM model from the pipeline
final_model_tss = grid_search_tss.best_estimator_.named_steps['lgbm']


Fitting 5 folds for each of 450 candidates, totalling 2250 fits
[CV] END lgbm__boosting_type=gbdt, lgbm__colsample_bytree=0.6, lgbm__min_child_weight=0.001, lgbm__n_estimators=100, lgbm__subsample=0.6; total time=  34.9s
[CV] END lgbm__boosting_type=gbdt, lgbm__colsample_bytree=0.6, lgbm__min_child_weight=0.001, lgbm__n_estimators=100, lgbm__subsample=0.6; total time=  59.6s
[CV] END lgbm__boosting_type=gbdt, lgbm__colsample_bytree=0.6, lgbm__min_child_weight=0.001, lgbm__n_estimators=100, lgbm__subsample=0.6; total time= 1.4min
[CV] END lgbm__boosting_type=gbdt, lgbm__colsample_bytree=0.6, lgbm__min_child_weight=0.001, lgbm__n_estimators=100, lgbm__subsample=0.6; total time= 1.9min
[CV] END lgbm__boosting_type=gbdt, lgbm__colsample_bytree=0.6, lgbm__min_child_weight=0.001, lgbm__n_estimators=100, lgbm__subsample=0.6; total time= 2.3min
[CV] END lgbm__boosting_type=gbdt, lgbm__colsample_bytree=0.6, lgbm__min_child_weight=0.001, lgbm__n_estimators=100, lgbm__subsample=0.8; total time=  

# hyperopt

In [17]:
#import required packages
from hyperopt import hp, tpe, Trials, STATUS_OK
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample
import gc #garbage collection
#optional but advised
import warnings
warnings.filterwarnings('ignore')

#GLOBAL HYPEROPT PARAMETERS
NUM_EVALS = 1000 #number of hyperopt evaluation rounds
N_FOLDS = 5 #number of cross-validation folds on data in each evaluation round

#LIGHTGBM PARAMETERS
LGBM_MAX_LEAVES = 2**9 #maximum number of leaves per tree for LightGBM
LGBM_MAX_DEPTH = 25 #maximum tree depth for LightGBM
EVAL_METRIC_LGBM_REG = 'mae' #LightGBM regression metric. Note that 'rmse' is more commonly used 
EVAL_METRIC_LGBM_CLASS = 'auc'#LightGBM classification metric

def quick_hyperopt(data, labels, num_evals=NUM_EVALS, Class=True, cat_features=None):
    
    print('Running {} rounds of LightGBM parameter optimisation:'.format(num_evals))
    #clear space
    gc.collect()

    integer_params = ['max_depth',
                     'num_leaves',
                      'max_bin',
                     'min_data_in_leaf',
                     'min_data_in_bin']

    def objective(space_params):

        #cast integer params from float to int
        for param in integer_params:
            space_params[param] = int(space_params[param])

        #extract nested conditional parameters
        if space_params['boosting']['boosting'] == 'goss':
            top_rate = space_params['boosting'].get('top_rate')
            other_rate = space_params['boosting'].get('other_rate')
            #0 <= top_rate + other_rate <= 1
            top_rate = max(top_rate, 0)
            top_rate = min(top_rate, 0.5)
            other_rate = max(other_rate, 0)
            other_rate = min(other_rate, 0.5)
            space_params['top_rate'] = top_rate
            space_params['other_rate'] = other_rate

        subsample = space_params['boosting'].get('subsample', 1.0)
        space_params['boosting'] = space_params['boosting']['boosting']
        space_params['subsample'] = subsample

        if Class:                
            if cat_features is not None:
                cv_results = lgb.cv(space_params, train, nfold = N_FOLDS, stratified=True, categorical_feature=cat_features,
                                    early_stopping_rounds=100, metrics=EVAL_METRIC_LGBM_CLASS, seed=42)
                best_loss = 1 - cv_results['auc-mean'][-1]
            else:
                cv_results = lgb.cv(space_params, train, nfold = N_FOLDS, stratified=True,
                                    early_stopping_rounds=100, metrics=EVAL_METRIC_LGBM_CLASS, seed=42)
                best_loss = 1 - cv_results['auc-mean'][-1]

        else:
            if cat_features is not None:
                cv_results = lgb.cv(space_params, train, nfold = N_FOLDS, stratified=False,
                                    early_stopping_rounds=100, metrics=EVAL_METRIC_LGBM_REG, seed=42)
                best_loss = cv_results['l1-mean'][-1] #'l2-mean' for rmse
            else:
                cv_results = lgb.cv(space_params, train, nfold = N_FOLDS, stratified=True,
                                    early_stopping_rounds=100, metrics=EVAL_METRIC_LGBM_REG, seed=42)
                best_loss = 1 - cv_results['auc-mean'][-1]

        return{'loss':best_loss, 'status': STATUS_OK }

    if cat_features is not None:
        train = lgb.Dataset(data, labels, categorical_feature=cat_features)
    else:
         train = lgb.Dataset(data, labels)

    #integer and string parameters, used with hp.choice()
    boosting_list = [{'boosting': 'gbdt',
                      'subsample': hp.uniform('subsample', 0.5, 1)},
                     {'boosting': 'goss',
                      'subsample': 1.0,
                     'top_rate': hp.uniform('top_rate', 0, 0.5),
                     'other_rate': hp.uniform('other_rate', 0, 0.5)}] #if including 'dart', make sure to set 'n_estimators'

    if Class:
        metric_list = ['auc'] #modify as required for other classification metrics
        objective_list = ['binary', 'cross_entropy']

    else:
        metric_list = ['MAE', 'RMSE'] 
        objective_list = ['huber', 'gamma', 'fair', 'tweedie']


    space ={'boosting' : hp.choice('boosting', boosting_list),
            'num_leaves' : hp.quniform('num_leaves', 2, LGBM_MAX_LEAVES, 1),
            'max_depth': hp.quniform('max_depth', 2, LGBM_MAX_DEPTH, 1),
            'max_bin': hp.quniform('max_bin', 32, 255, 1),
            'min_data_in_leaf': hp.quniform('min_data_in_leaf', 1, 256, 1),
            'min_data_in_bin': hp.quniform('min_data_in_bin', 1, 256, 1),
            'min_gain_to_split' : hp.quniform('min_gain_to_split', 0.1, 5, 0.01),
            'lambda_l1' : hp.uniform('lambda_l1', 0, 5),
            'lambda_l2' : hp.uniform('lambda_l2', 0, 5),
            'learning_rate' : hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
            'metric' : hp.choice('metric', metric_list),
            'objective' : hp.choice('objective', objective_list),
            'feature_fraction' : hp.quniform('feature_fraction', 0.5, 1, 0.01),
            'bagging_fraction' : hp.quniform('bagging_fraction', 0.5, 1, 0.01)
        }

    trials = Trials()
    best = fmin(fn=objective,
                space=space,
                algo=tpe.suggest,
                max_evals=num_evals, 
                trials=trials)

    #fmin() will return the index of values chosen from the lists/arrays in 'space'
    #to obtain actual values, index values are used to subset the original lists/arrays
    best['boosting'] = boosting_list[best['boosting']]['boosting']#nested dict, index twice
    best['metric'] = metric_list[best['metric']]
    best['objective'] = objective_list[best['objective']]

    #cast floats of integer params to int
    for param in integer_params:
        best[param] = int(best[param])

    print('{' + '\n'.join('{}: {}'.format(k, v) for k, v in best.items()) + '}')
    return(best)    

In [ ]:
# Specify the column name for the categorical feature in a list
# TODO: change the imbalance_buy_sell_flag from -1, 0, 1 to 0, 1, 2
# cat_features = ['imbalance_buy_sell_flag']
#
# Call quick_hyperopt function
best_params = quick_hyperopt(X, y, num_evals=1000, Class=False)

print("Optimized hyperparameters:", best_params)

# First Version

In [ ]:
# Train model
model = make_pipeline(
    ImbalanceCalculator, 
    LGBMRegressorCV(params = {'random_state': seed}, cv=tss)
)
model.fit(X_train, y_train)

In [ ]:
lgbm_models = model.named_steps['lgbmregressorcv'].models

# Inference and Submission

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    feat = imbalance_calculator(test.drop('row_id', axis = 1))
    
    sample_prediction['target'] = np.mean([model.predict(feat) for model in lgbm_models], 0)
    np.mean([model.predict(feat) for model in lgbm_models], 0)
    env.predict(sample_prediction)
    counter += 1